Implementation of the metaheuristics algo on a knapsack problem

In [136]:
import numpy as np


In [312]:
# defining the Knapsak problem

np.random.seed(42)  # For reproducibility

num_items = 100
V = np.random.randint(1, 100, size=num_items)  # Random values for the items
W = np.random.randint(1, 100, size=num_items)  # Random weights for the items
W_Max = 500
X_0=np.zeros(num_items)

def kp_value(V,X): # returning value of the knapsak
    return np.dot(V,X)

X=np.random.randint(0, 2, size=num_items)
print(X)
print(kp_value(V,X))

def Isfeasible(W_max,W,X): # testing if X is a feasible solution
    return True if np.dot(W,X)<=W_max else False

def kp_weight(W,X): # returning weight of the knapsak
    return np.dot(W,X)


print(Isfeasible(W_max,W,X))
print(kp_weight(W,X))

[0 0 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0
 1 0 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 1 0 1 1
 0 0 1 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0]
2625
False
2382


In [315]:
# Méthode de descente simple

def Neighbours(X,W,W_Max): # Neighbours function returning feasible neighbours - neigbour is defined as one article more or less than in X so len(X) neighbours in total
    N=[]
    n=len(X)
    for k in range(n):
        X_N=X.copy()
        X_N[k]=(X[k]+1) % 2
        if Isfeasible(W_Max,W,X_N)==True:
            N.append(X_N)
    return N


def descente_simple(V,W,W_Max,X_0=X_0):
    i=0
    Values=[]
    V_0=kp_value(V,X_0)
    Values.append(V_0) # keep track of values at each iteration
    X=[]
    X.append(X_0)
    
    while True:
        # print("iteration:",i)
        N=Neighbours(X[i],W,W_Max) # construction of the list of neighbours at iteration "i"
        V_max=0
        
        for n in N: # iteration over all neighbours
             if kp_value(V,n) >= V_max:
                V_max=kp_value(V,n)
                n_max=n
        
        if V_max <=Values[i]:
            break

        Values.append(V_max)
        X.append(n_max)
        i+=1

    return Values,X

Values,X=descente_simple(V,W,W_Max)

print("Best_value:",Values[-1]) # best value is necessarly the last one
print("Best_X:",X[-1])




Best_value: 1050.0
Best_X: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [316]:
# Recuit simulé / Simulated annealing

import random
import math

def recuit_simule(V,W,W_Max,X_0=X_0,T=2,alpha=0.90,j_max=50):
    i=0
    Values=[]
    V_0=kp_value(V,X_0)
    Values.append(V_0)
    X=[]
    X.append(X_0)
    V_max=0 # need to keep track of the max value seen so far
    X_max=X_0
    outer_break = False

    while not outer_break:
        j=0
        # print("iteration =", i)
        N=Neighbours(X[i],W,W_Max)
        V_current=0
        nb=len(N) # size of neighbours
        
        while True:
            k=random.randint(0,nb-1) # one neighbour is selected randomly
            if random.uniform(0,1) < np.exp(-(Values[i-1]-kp_value(V,N[k]))/T): # criteria to accept or not the new solution depending on relative value to previous solution and temperature
                    V_current=kp_value(V,N[k])
                    X_current=N[k]
                    break
            else:
                j+=1
                if j>=j_max: # we break the code if no solution has been accepted after j_max tries
                    outer_break=True

        if V_current > V_max: # we update best value if successful
            V_max=V_current
            X_max=X_current
        
        Values.append(V_current) # we keep track of best value at each iteration
        X.append(X_current)
        T=alpha*T
        i+=1

    return Values,X,V_max,X_max


Values,X,V_max,X_max=recuit_simule(V,W,W_max)
print("Best_value:",V_max)
print("Best_X:",X_max)

Best_value: 1428.0
Best_X: [1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 0.]


C:\Users\felix\AppData\Local\Temp\ipykernel_27968\2850803892.py:26: RuntimeWarning: overflow encountered in exp
  if random.uniform(0,1) < np.exp(-(Values[i-1]-kp_value(V,N[k]))/T): # criteria to accept or not the new solution depending on relative value to previous solution and temperature


In [319]:
# Tabou

def tabou(V,W,W_max,X_0=X_0,j_max=100):
    i=0
    j=0
    Values=[]
    V_0=kp_value(V,X_0)
    Values.append(V_0)
    X=[]
    X.append(X_0)
    Tabou=[] # initialization of list of tabou solutions
    V_max=0

    while True:
        # print("iteraion =", i)
        N=Neighbours(X[i],W,W_max)
        if not N: # break if neighbour is empty
            break
        V_current=0
        nb=len(N)

        for sol in N:
            if any(np.array_equal(sol, tabou_item) for tabou_item in Tabou):
                continue # we do not consider this solution as it is tabou
            elif kp_value(V,sol) >= V_current:
                V_current=kp_value(V,sol)
                X_current=sol
            
        Values.append(V_current)
        X.append(X_current)
        Tabou.append(X_current)
        
        if V_current >V_max:
            V_max=V_current
            X_max=X_current
        else:
            j+=1
            if j==j_max: # we break the code if no better solution has been found after j_max tries
                break

        i+=1

    return Values,X,V_max,X_max,Tabou

Values,X,V_max,X_max,Tabou=tabou(V,W,W_max)
print("Best_value:",V_max)
print("Best_X:",X_max)
# print("Tabou:",Tabou)
print("Number of tabou solutions:",len(Tabou))



Best_value: 1253.0
Best_X: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0.]
Number of tabou solutions: 116


In [320]:
# Fourmi

def Force_gloutonne(V,W): # fonction qui calcule la force gloutonne (qualité intrinsèque d'une solution)
    n=len(V)
    Force_gloutonne=np.ones((2,n)) # première ligne correspond aux choix "0" (article non sélectionné), deuxième ligne correspond aux choix "1" (article sélectionné)
    ratio_V_W=np.array(V)/np.array(W) # ratio valeur / poids
    for i in range(n):
        Force_gloutonne[1,i] = (ratio_V_W[i]-np.min(ratio_V_W))/(np.max(ratio_V_W)-np.min(ratio_V_W))
        Force_gloutonne[0,i] = 1- Force_gloutonne[1,i] 
    return Force_gloutonne

def Generation(taille,Trace,Force_gloutonne,alpha,beta): # fonction qui génère une nouvelle génération de solutions
    size = Trace.shape[1]
    numerator = np.power(Force_gloutonne,alpha) * np.power(Trace,beta)
    probabilities = numerator / (np.sum(numerator,axis=0))
    values = [0,1]
    SolutionS=[]
    i=1
    while i <= taille:
        sol=np.zeros(size, dtype=int)
        for j in range(size):
            partial_solution=np.random.choice(values, size=1, p=probabilities[:,j].ravel())
            sol[j]=partial_solution[0]
        sol_list = list(sol)
        if Isfeasible(W_Max, W, sol_list) and sol_list not in SolutionS:
            SolutionS.append(sol_list)
            i+=1
    return SolutionS


def update_trace(Trace,Curent_Generation,decay=0.1,c=1): # fonction qui update la trace
    Delta=np.zeros((Trace.shape))
    for fourmi in Curent_Generation:
        for i in range(len(fourmi)):
            if fourmi[i]==0:
                Delta[0,i]+=kp_value(V,fourmi)
            else:
                Delta[1,i]+=kp_value(V,fourmi)
    
    Trace=(1-decay)*Trace+c*Delta
    return Trace

def best_fourmi(Population): # fonction qui selectionne la meilleure solution/fourmi parmi une population donné
    V_max=0
    for fourmi in Population:
        if kp_value(V,fourmi)>V_max:
            V_max=kp_value(V,fourmi)
            X_max=fourmi
    return V_max,X_max

    
def fourmi(V,W,W_max,X_0=X_0,j_max=50,taille=5,alpha=0.5,beta=0.5):
    taille_X=len(X_0)
    j=0
    Trace=np.ones((2,taille_X))
    X_max=X_0
    V_max=kp_value(X_0,W)
    Force=Force_gloutonne(V,W)

    while True:
        Current_Generation=Generation(taille,Trace,Force,alpha,beta)
        c=1/(taille) # je ne sais pas trop comment chosir le c pour que la Trace ne croit pas trop vite rendant la force gloutonne négligeable
        Trace=update_trace(Trace,Current_Generation,c=c)
        j+=1
        V_current,X_current=best_fourmi(Current_Generation)
        if V_current>V_max:
            V_max=V_current
            X_max=X_current

        if j==j_max:
                break

    return V_max,X_max,Current_Generation,Trace,Force

V_max,X_max,Last_Generation,Trace,Force = fourmi(V,W,W_max)
print("Best_value:",V_max)
print("Best_X:",X_max)
print("Last Generation:", Last_Generation)
print("Trace:",Trace)
print("Force:",Force)

Best_value: 1129
Best_X: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
Last Generation: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 